<a href="https://colab.research.google.com/github/alexrofail/Loss-Optimizers-Training-Loops/blob/main/callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch.nn.functional as F
import gzip
import pickle
from torch import tensor
from torch import nn

In [2]:
import pickle, gzip, math, torch, matplotlib as mpl

In [3]:
def accuracy(out, yb):
  return (torch.argmax(out, dim=1)==yb).float().mean()

In [8]:
from torch import optim
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler

In [5]:
class Dataset():
  def __init__(self,x,y):
    self.x ,self.y = x,y
  
  def __len__(self):
    return len(self.x)

  def __getitem__(self,i):
    return self.x[i], self.y[i]

In [9]:
def get_dls(train_ds, valid_ds, bs, **kwargs):
  return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
          DataLoader(valid_ds, batch_size=bs*2, **kwargs))

In [10]:
#TODO: very important, go back and fix the exports thing 
#TODO: cont... so that you don't have to c/p code above to maintain library